# Bestimmung der Assoziationsregeln
In den vorangegangenen Abschnitten haben wir aus Transaktionsdatenbanken häufige Itemsets hergeleitet. Aus diesen sollen nun Assoziationsregeln generiert werden.

In [ ]:
from tui_dsmt.fpm import Itemset

import pandas as pd
from mlxtend.frequent_patterns import association_rules

## Inhaltsverzeichnis
- [Wiederholung](#Wiederholung)
- [Algorithmus](#Algorithmus)
- [Weitere Interessantheitsmaße](#Weitere-Interessantheitsmaße)
- [Implementierungen](#Implementierungen)

## Wiederholung

Eine Assoziationsregel ist eine Regel in der folgenden Form.
$$
\text{Rumpf} \rightarrow \text{Kopf} [\text{support}, \text{confidence}]
$$

**Rumpf** und **Kopf** sind disjunkte Itemsets beliebiger Größe.

Der **Support einer Assoziationsregel** $X \Rightarrow Y$ ist der Anteil des gemeinsamen Auftretens von $X$ und $Y$ in Relation zur Anzahl aller Transaktionen: $$
s(X \Rightarrow Y) = \frac{\sigma(X \cup Y)}{|T|}
$$

Die **Konfidenz einer Assoziationsregel** $X \Rightarrow Y$ ist der Anteil des gemeinsamen Auftretens von $X$ und $Y$ in Relation zur Anzahl des Auftretens von $X$: $$
c(X \Rightarrow Y) = \frac{\sigma(X \cup Y)}{\sigma(X)}
$$

Nachfolgend verwenden wir den Support eines Itemsets in Relation zur Gesamtgröße der Transaktionsdatenbank. Der Divisor $\left| T \right|$ entfällt daher bei der Berechnung des Supports einer Assoziationsregel!

## Algorithmus
Der Algorithmus zum Auffinden von Assoziationsregeln wurde bereits zu Beginn dieses Kapitels angesprochen. Zuerst werden dafür die häufigen Itemsets gesucht und zusammen mit ihrem Support in einem Dictionary gespeichert, um Datenbankzugriffe zu vermeiden.

Da laut Monotonie-Kriterium alle Teilmengen eines häufigen Itemsets ebenfalls häufig sein müssen, findet sich der Support jeder Teilmenge eines häufigen Itemsets ebenfalls in diesem Dictionary.

In [ ]:
frequent_itemsets = {
    Itemset('A'): 0.15,
    Itemset('B'): 0.2,
    Itemset('C'): 0.25,
    Itemset('A', 'B'): 0.15,
    Itemset('A', 'C'): 0.1,
    Itemset('B', 'C'): 0.2,
    Itemset('A', 'B', 'C'): 0.1
}

Für jedes häufige Itemset werden alle *echten*, *nichtleeren* Teilmengen $\emptyset \neq A \subset X$ betrachtet. Geprüft wird dann, ob die Assoziationsregel $A \rightarrow (X \setminus A)$ eine vorgegebene Konfidenz erfüllt.

In [ ]:
for X in frequent_itemsets:
    print(f'{X=}')
    for A in X.real_nonempty_subsets:
        c = frequent_itemsets[X] / frequent_itemsets[X-A]
        print(f'    {A=:<6}  ->  {X-A=:<6}    {c=:.3f}')

$1$-Itemsets entfallen in dieser Betrachtung, da die leere Menge die einzige echte Teilmenge einer einelementigen Menge ist. Die Berechnung der Konfidenz kann außerdem vereinfacht werden, da folgendes gilt:

$$
c(A \Rightarrow X \setminus A) = \frac{\sigma(A \cup (X \setminus A))}{\sigma(A)} = \frac{\sigma(X)}{\sigma(A)}
$$

## Weitere Interessantheitsmaße
Neben der Konfidenz existieren noch weitere Interessantheitsmaße, um eine Assoziationsregel zu bewerten.

Der **Lift** einer Assoziationsregel $X \Rightarrow Y$ wird folgendermaßen berechnet:

$$
lift(X \Rightarrow Y) = \frac{\sigma(X \cup Y)}{\sigma(X) * \sigma(Y)} = \frac{c(X \Rightarrow Y)}{\sigma(Y)}
$$

Er gibt an, wie viel häufiger $X$ und $Y$ gemeinsam auftreten als es bei statistischer Unabhängigkeit zu erwarten wäre. Der Wert ist
- $=1$, wenn Itemsets unabhängig voneinander gekauft werden.
- $>1$, wenn es eine positive Assoziation zwischen den Itemsets gibt bzw. sie häufiger zusammen gekauft werden, als es zufällig zu erwarten wäre.
- $<1$, wenn es eine negative Assoziation zwischen den Itemsets gibt bzw. sie seltener zusammen gekauft werden, als es zufällig zu erwarten wäre.

Der **Leverage** / Einfluss einer Assoziationsregel $X \Rightarrow Y$ wird folgendermaßen berechnet:

$$
leverage(X \Rightarrow Y) = s(X \Rightarrow Y) - \sigma(X) * \sigma(Y) = \sigma(X \cup Y) - \sigma(X) * \sigma(Y)
$$

Der Leverage berechnet damit die Differenz der Häufigkeit des gemeinsamen Auftretens gegenüber der Häufigkeit des Auftretens bei statistischer Unabhängigkeit. Der Wert ist
- $=0$, falls die Itemsets genauso häufig zusammen auftreten wie es zufällig zu erwarten wäre.
- $>0$, falls die Itemsets häufiger zusammen auftreten als es zufällig zu erwarten wäre.
- $<0$, falls die Itemsets seltener zusammen auftreten als es zufällig zu erwarten wäre.

In [ ]:
for X in frequent_itemsets:
    print(f'{X=}')
    for A in X.real_nonempty_subsets:
        c = frequent_itemsets[X] / frequent_itemsets[A]
        lift = c / frequent_itemsets[X-A]
        leverage = frequent_itemsets[X] - frequent_itemsets[A] * frequent_itemsets[X-A]

        print(f'    {A:<6} -> {X-A:<6}    {c=:.3f} {lift=:.3f} {leverage=:.3f}')

## Implementierungen
Eine fertige Implementierung zur Berechnung von Assoziationsregeln und deren Bewertung steht beispielsweise im Paket `mlxtend` bereit. Dazu müssen die häufigen Itemsets jedoch zunächst in ein Pandas DataFrame konvertiert werden.

In [ ]:
fi_df = pd.DataFrame({
    'support': (v for v in frequent_itemsets.values()),
    'itemsets': frequent_itemsets.keys()
})
fi_df

Anschließend können die Assoziationsregeln und eine Vielzahl an Interessantheitsmaßen mit der Funktion `association_rules` bestimmt werden.

In [ ]:
association_rules(fi_df,
                  num_itemsets=len(fi_df),
                  metric='confidence',
                  min_threshold=0.0)